<a href="https://colab.research.google.com/github/Rithvik50/DL-Project/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-community langchain-text-splitters langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install -qU chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.2 MB/s eta 

# LOAD THE DOCUMENTS

In [3]:
import os
import re
from typing import List, Dict
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

class NutritionalDataProcessor:
    def __init__(self, data_dir: str = ".", collection_name: str = "nutrition_data"):
        """
        Initialize the processor with a data directory and collection name
        """
        self.data_dir = data_dir
        self.collection_name = collection_name
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        self.embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    def load_data_from_directory(self) -> List[str]:
        """
        Load all text files from the specified directory
        """
        text_data = []
        for filename in os.listdir(self.data_dir):
            if filename.endswith('.txt') or filename.endswith('.md'):
                with open(os.path.join(self.data_dir, filename), 'r', encoding='utf-8') as file:
                    text_data.append(f"\n\n=== FILE: {filename} ===\n{file.read()}")
        print(f"Loaded {len(text_data)} files from {self.data_dir}")
        return text_data

    def _split_into_sections(self, text: str) -> Dict[str, str]:
        """
        Split text into sections based on predefined patterns
        """
        sections = {}
        section_patterns = [
            (r"Dietary Reference Intakes for Minerals", "DRI_Minerals"),
            (r"Dietary Reference Intakes for Vitamins and Macronutrients", "DRI_Vitamins_Macros"),
            (r"Estimated Average Requirements \(EAR\)", "EAR"),
            (r"Tolerable Upper Intake Levels \(TUL\)", "TUL")
        ]
        current_section = "Introduction"
        current_content = []
        lines = text.split('\n')
        for line in lines:
            section_found = False
            for pattern, section_name in section_patterns:
                if re.search(pattern, line, re.IGNORECASE):
                    if current_content:
                        sections[current_section] = '\n'.join(current_content)
                    current_section = section_name
                    current_content = [line]
                    section_found = True
                    break
            if not section_found:
                current_content.append(line)
        if current_content:
            sections[current_section] = '\n'.join(current_content)
        print(f"Detected sections: {list(sections.keys())}")
        return sections

    def _is_demographic_group(self, line: str) -> bool:
        """
        Check if a line represents a demographic group header
        """
        demographic_pattern = r"^(?:#+\s+)?(?:Infants|Children|Males|Females|Pregnancy|Lactation)(?:,)?\s*(?:\d+[-–]\d+(?:\+)?\s*(?:y|mo|Years)|>\s*\d+\s*(?:y|Years))"
        print(f"Checking line: '{line.strip()}', Match: {bool(re.match(demographic_pattern, line.strip(), re.IGNORECASE))}")
        return bool(re.match(demographic_pattern, line.strip(), re.IGNORECASE))

    def _process_dri_section(self, content: str, section_type: str) -> List[Document]:
        """
        Process a DRI section into documents with demographic metadata
        """
        documents = []
        current_group = None
        current_nutrients = []
        seen_groups = set()  # Track processed groups to avoid duplicates
        lines = content.split('\n')
        for line in lines:
            line = line.strip()
            if not line:
                continue
            if self._is_demographic_group(line):
                if current_group and current_nutrients and current_group not in seen_groups:
                    doc = self._create_demographic_document(current_group, current_nutrients, section_type)
                    documents.append(doc)
                    seen_groups.add(current_group)
                    print(f"Created document for group: {current_group}, Section: {section_type}")
                current_group = line.strip().lstrip('#').strip()
                current_nutrients = []
            elif ':' in line and any(unit in line for unit in ['mg/d', 'μg/d', 'µg/d', 'g/d', 'L/d']):
                print(f"Parsed nutrient line: {line}")
                current_nutrients.append(line)
        if current_group and current_nutrients and current_group not in seen_groups:
            doc = self._create_demographic_document(current_group, current_nutrients, section_type)
            documents.append(doc)
            seen_groups.add(current_group)
            print(f"Created document for group: {current_group}, Section: {section_type}")
        return documents

    def _extract_metadata(self, group: str, section_type: str, nutrients: List[str]) -> Dict:
        """
        Extract metadata for a document
        """
        nutrient_count = len([n for n in nutrients if ':' in n])
        return {
            "demographic_group": group,
            "section_type": section_type,
            "nutrient_count": nutrient_count,
            "document_type": "general"
        }

    def _create_demographic_document(self, group: str, nutrients: List[str], section_type: str) -> Document:
        """
        Create a LangChain Document for a demographic group
        """
        content = f"Demographic Group: {group}\n"
        content += f"Reference Type: {section_type}\n\n"
        content += "Nutritional Requirements:\n"
        for nutrient in nutrients:
            content += f"- {nutrient}\n"
        metadata = self._extract_metadata(group, section_type, nutrients)
        return Document(page_content=content, metadata=metadata)

    def parse_nutritional_data(self, texts: List[str]) -> List[Document]:
        """
        Parse nutritional data into documents
        """
        all_documents = []
        for text in texts:
            sections = self._split_into_sections(text)
            for section_name, content in sections.items():
                if section_name in ["DRI_Minerals", "DRI_Vitamins_Macros", "EAR", "TUL"]:
                    documents = self._process_dri_section(content, section_name)
                    all_documents.extend(documents)
                else:
                    chunks = self.text_splitter.split_text(content)
                    for i, chunk in enumerate(chunks):
                        metadata = {
                            "section": section_name,
                            "chunk_id": i,
                            "document_type": "general"
                        }
                        all_documents.append(Document(page_content=chunk, metadata=metadata))
        print(f"Created {len(all_documents)} documents")
        return all_documents

    def create_vectorstore(self, documents: List[Document], collection_name: str = None) -> Chroma:
        """
        Create a Chroma vector store from documents
        """
        collection_name = collection_name or self.collection_name
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embedding_function,
            collection_name=collection_name,
        )
        print(f"Created vector store with {len(documents)} documents in collection '{collection_name}'")
        return vectorstore

# MODEL IMPLEMENTATION

In [4]:
!pip install -qU langchain-perplexity

In [5]:
from langchain_perplexity import ChatPerplexity
from google.colab import userdata

pplx_api_key = userdata.get('PERPLEX_API_KEY')

llm_1 = ChatPerplexity(pplx_api_key=pplx_api_key, model="sonar", temperature=0.0)

In [6]:
from typing import List, Dict, Any
import re
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document
from langchain_perplexity import ChatPerplexity

class PerplexityRAGQuery:
    def __init__(self, vectorstore, perplexity_api_key: str):
        """
        Initialize with vector store and Perplexity API key
        """
        self.vectorstore = vectorstore
        self.client = ChatPerplexity(pplx_api_key=perplexity_api_key, model="sonar", temperature=0.0)

        self.prompt_template = ChatPromptTemplate.from_template(
            """
            You are a nutrition assistant. Use only the provided context to construct a nutrient requirement list.

            Instructions:
            1. Use ONLY the context provided, which is filtered for the specific demographic group.
            2. Output ONLY a clean list of nutrients and their values, one per line, in the format: "Nutrient Name: Value [TUL : TUL Value]"
            3. Do not include any bullet points, dashes, asterisks, or markdown formatting.
            4. Do not include any introductions, conclusions, explanations, or additional text.
            5. Do not include section headers or document references.
            6. If no relevant data is found, return "Data not found".
            7. Make sure the values do not exceed specified TULs.
            8. The lives of patients are in your hands. False information leads to serious health impairment or death.

            Context:
            {context}

            User Query:
            {question}

            Output format example:
            Calcium: 1000 mg/d [TUL : 2500 mg/d]
            Iron: 8 mg/d [TUL : 45 mg/d]
            Vitamin C: 90 mg/d [TUL : 2000 mg/d]
            """
        )

    def extract_demographic_from_query(self, query: str) -> tuple[int, str]:
        """
        Extract age and sex from the query using regex
        """
        age_pattern = r'(\d{1,3})\s*(?:year(?:s)?(?:\s|-)?old)?'
        age_match = re.search(age_pattern, query, re.IGNORECASE)
        age = int(age_match.group(1)) if age_match else None

        sex_pattern = r'\b(male|man|female|woman)\b'
        sex_match = re.search(sex_pattern, query, re.IGNORECASE)
        sex = sex_match.group(1).lower() if sex_match else None
        sex = 'male' if sex in ['man', 'male'] else 'female' if sex in ['woman', 'female'] else None

        return age, sex

    def get_target_headers(self, age: int, sex: str) -> List[str]:
        """
        Get standardized target headers for the demographic
        """
        if age < 1:
            return ["Infants 0-6 months", "Infants 7-12 months"]
        elif 1 <= age <= 3:
            return ["Children 1-3 years"]
        elif 4 <= age <= 8:
            return ["Children 4-8 years"]
        elif 9 <= age <= 13:
            return [f"{sex.capitalize()}s 9-13 years"]
        elif 14 <= age <= 18:
            return [f"{sex.capitalize()}s 14-18 years"]
        elif 19 <= age <= 30:
            headers = [f"{sex.capitalize()}s 19-30 years"]
            headers.append(f"{sex.capitalize()}s 19-70+ years")
            return headers
        elif 31 <= age <= 50:
            headers = [f"{sex.capitalize()}s 31-50 years"]
            headers.append(f"{sex.capitalize()}s 19-70+ years")
            return headers
        elif 51 <= age <= 70:
            headers = [f"{sex.capitalize()}s 51-70 years"]
            headers.append(f"{sex.capitalize()}s 19-70+ years")
            return headers
        else:  # > 70
            headers = [f"{sex.capitalize()}s > 70 years"]
            headers.append(f"{sex.capitalize()}s 19-70+ years")
            return headers

    def retrieve_relevant_documents(self, query: str, age: int = None, sex: str = None, k: int = 10) -> List[Document]:
        """
        Retrieve documents for the demographic from all relevant sections
        """
        if age is None or sex is None:
            age, sex = self.extract_demographic_from_query(query)

        if not age or not sex:
            return []

        target_headers = self.get_target_headers(age, sex)
        sections = ["DRI_Minerals", "DRI_Vitamins_Macros", "EAR", "TUL"]

        all_documents = []

        for section in sections:
            for header in target_headers:
                filters = {
                    "$and": [
                        {"demographic_group": {"$eq": header}},
                        {"section_type": {"$eq": section}}
                    ]
                }

                try:
                    docs = self.vectorstore.similarity_search(
                        query,
                        k=k,
                        filter=filters
                    )

                    validated_docs = [
                        doc for doc in docs
                        if (doc.metadata.get("demographic_group") == header and
                            doc.metadata.get("section_type") == section)
                    ]

                    all_documents.extend(validated_docs)

                    if validated_docs:
                        break

                except Exception:
                    continue

        unique_docs = []
        seen_content = set()
        for doc in all_documents:
            content_hash = hash(doc.page_content)
            if content_hash not in seen_content:
                unique_docs.append(doc)
                seen_content.add(content_hash)

        return unique_docs

    def format_context(self, documents: List[Document]) -> str:
        """
        Format retrieved documents into context string
        """
        context_parts = []
        for i, doc in enumerate(documents):
            context_part = f"Document {i+1} [{doc.metadata.get('demographic_group', 'Unknown')} - {doc.metadata.get('section_type', 'Unknown')}]\n{doc.page_content}\n"
            context_parts.append(context_part)
        return "\n".join(context_parts)

    def query_perplexity(self, prompt: str, model: str = "sonar") -> str:
        """
        Query Perplexity API with the formatted prompt
        """
        try:
            response = self.client.invoke(prompt)
            return response.content
        except Exception:
            return ""

    def answer_query(self, question: str, age: int = None, sex: str = None, model: str = "sonar") -> str:
        """
        Complete RAG pipeline: retrieve, format, and query
        """
        if age is None or sex is None:
            age, sex = self.extract_demographic_from_query(question)

        if not age or not sex:
            return f"The ideal nutritional intake for a {age or 'unknown'} year old {sex or 'unknown'} is as follows:\n"

        documents = self.retrieve_relevant_documents(question, age, sex)

        if not documents:
            return f"The ideal nutritional intake for a {age} year old {sex} is as follows:\n"

        context = self.format_context(documents)

        formatted_prompt = self.prompt_template.format(
            context=context,
            question=question
        )

        nutrient_list = self.query_perplexity(formatted_prompt, model)

        return f"The ideal nutritional intake for a {age} year old {sex} is as follows:\n{nutrient_list}"


def test_nutrition_query(processor, vectorstore, perplexity_api_key: str):
    """
    Test function with the specified query
    """
    rag_system = PerplexityRAGQuery(vectorstore, perplexity_api_key)
    age = int(input("Enter the user's age :"))
    sex = input("Enter the user's sex (male/female) :")
    test_query = f"What are the ideal nutritional requirements for a {age} year old {sex}?"
    result = rag_system.answer_query(question=test_query)
    print(result)
    return result

In [7]:
# First LLM setup (PerplexityRAGQuery)
processor = NutritionalDataProcessor(data_dir=".")
texts = processor.load_data_from_directory()
documents = processor.parse_nutritional_data(texts)
vectorstore = processor.create_vectorstore(documents)
rag_system = PerplexityRAGQuery(vectorstore, pplx_api_key)

/tmp/ipython-input-3-1454532418.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded 1 files from .
Detected sections: ['Introduction', 'DRI_Minerals', 'DRI_Vitamins_Macros', 'EAR', 'TUL']
Checking line: '# Dietary Reference Intakes for Minerals', Match: False
Checking line: '## Infants 0-6 months', Match: True
Checking line: '- **Calcium**: 200 mg/d', Match: False
Parsed nutrient line: - **Calcium**: 200 mg/d
Checking line: '- **Chromium**: 0.2 μg/d', Match: False
Parsed nutrient line: - **Chromium**: 0.2 μg/d
Checking line: '- **Copper**: 200 μg/d', Match: False
Parsed nutrient line: - **Copper**: 200 μg/d
Checking line: '- **Fluoride**: 0.01 mg/d', Match: False
Parsed nutrient line: - **Fluoride**: 0.01 mg/d
Checking line: '- **Iodine**: 110 μg/d', Match: False
Parsed nutrient line: - **Iodine**: 110 μg/d
Checking line: '- **Iron**: 0.27 mg/d', Match: False
Parsed nutrient line: - **Iron**: 0.27 mg/d
Checking line: '- **Magnesium**: 30 mg/d', Match: False
Parsed nutrient line: - **Magnesium**: 30 mg/d
Checking line: '- **Manganese**: 0.003 mg/d', Match: False

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Created vector store with 80 documents in collection 'nutrition_data'


In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_perplexity import ChatPerplexity

# Second LLM setup (Medical Nutrition Advisor)
medical_prompt = ChatPromptTemplate.from_template(
    """
    You are a clinical nutrition specialist AI. Analyze the user's medical profile and adjust baseline nutrient requirements using evidence-based clinical guidelines.

--- Baseline Nutritional Values ---
{baseline_values}

--- User Profile ---
• Health Conditions: {conditions}
• Activity Level: {activity_level}
• Weight: {weight_kg} kg
• Height: {height_cm} cm
• Medications: {medications}

--- Clinical Decision Framework ---

**STEP 1: CONDITION ANALYSIS**
For each condition, identify specific nutrient modifications:
- Example: "Iron deficiency anemia" → Increase iron 2-3x, add vitamin C for absorption
- Example: "Hypertension" → Reduce sodium to <2000mg/d, increase potassium/magnesium

**STEP 2: MEDICATION INTERACTIONS**
Check each medication for nutrient interactions:
- Example: "Warfarin" → Maintain consistent vitamin K intake, avoid large fluctuations
- Example: "Furosemide" → Monitor potassium, magnesium depletion risk

**STEP 3: WEIGHT- AND HEIGHT-BASED ADJUSTMENTS**
Use weight and height to adjust energy and macronutrient needs:
- **Energy Needs**: Estimate Total Daily Energy Expenditure (TDEE) using the Mifflin-St Jeor Equation for Basal Metabolic Rate (BMR), adjusted for activity level:
  - BMR (males) = 10 × weight (kg) + 6.25 × height (cm) - 5 × age (years) + 5
  - BMR (females) = 10 × weight (kg) + 6.25 × height (cm) - 5 × age (years) - 161
  - TDEE = BMR × Activity Factor (Sedentary: 1.2, Light: 1.375, Moderate: 1.55, Active: 1.725)
- **Protein**: Adjust based on weight and activity:
  - Sedentary: 0.8 g/kg
  - Light/Moderate: 1.0–1.4 g/kg
  - Active: 1.4–2.0 g/kg
- **Carbohydrates**: Adjust based on weight and activity (3–5 g/kg for light/moderate, 5–7 g/kg for active)
- **Total Water**: Adjust based on weight (30–40 mL/kg, increase by 500–1000 mL for moderate/active levels)
- Example: For a 70 kg male, 175 cm, moderate activity:
  - Protein: 70–98 g/day (1.0–1.4 g/kg)
  - Carbohydrates: 210–350 g/day (3–5 g/kg)
  - Water: 2.1–2.8 L/day + 500–1000 mL = ~2.6–3.8 L/day

**STEP 4: CONFLICT RESOLUTION**
When conditions or weight/height-based adjustments conflict:
1. STATE: "CLINICAL CONFLICT: [Condition A] requires [X], but [weight/height-based adjustment] requires [Y]"
2. PRIORITIZE: Based on immediate health risk (e.g., prioritize medical condition over activity-based increases)
3. COMPROMISE: Suggest middle ground with monitoring

**STEP 5: SAFETY CHECK**
- Flag any value >75% of TUL as "CAUTION"
- Flag any value >90% of TUL as "MEDICAL SUPERVISION REQUIRED"

**SPECIAL CONSIDERATION FOR MAGNESIUM:**
- The TUL of 350 mg/d applies ONLY to supplemental magnesium (pills, powders, fortified foods), NOT to magnesium naturally present in food
- The RDA (400-420 mg/d for adult males, 310-320 mg/d for adult females) represents safe total intake from all sources
- Only flag as "CAUTION" if supplemental magnesium alone exceeds 350 mg/d
- Example: 400 mg/d total (300 mg food + 100 mg supplements) = Safe
- Example: 400 mg/d total (50 mg food + 350 mg supplements) = At TUL limit, monitor for GI effects

--- Required Output Format ---

| Nutrient | Baseline | Adjusted | Change | Clinical Rationale | Risk Level |
|----------|----------|----------|--------|---------------------|------------|
[Complete table for all nutrients]

**CONFLICTS IDENTIFIED:** [List any conflicts and resolutions]
**HIGH-RISK NUTRIENTS:** [List nutrients requiring monitoring]

"""
)

llm_2 = ChatPerplexity(pplx_api_key=pplx_api_key, model="sonar-pro", temperature=0.0)

# Function to parse baseline values into a suitable format
def parse_baseline_values(result, config) -> dict:
    # Handle potential dictionary response (e.g., from ChatPerplexity)
    if isinstance(result, dict) and 'content' in result:
        result = result['content']
    elif not isinstance(result, str):
        raise ValueError(f"Unexpected result type: {type(result)}. Expected string or dict with 'content' key.")

    # Extract the nutrient list (remove the introductory sentence)
    nutrient_list = result.split('\n', 1)[1].strip()
    input_data = config["configurable"]["input_data"]
    return {
        "baseline_values": nutrient_list,
        "conditions": input_data["conditions"],
        "activity_level": input_data["activity_level"],
        "weight_kg": input_data["weight_kg"],
        "height_cm": input_data["height_cm"],
        "medications": input_data["medications"]
    }

# Runnable pipeline
chain = (
    # Step 1: Get baseline values from the first LLM
    RunnableLambda(lambda x: rag_system.answer_query(x["query"]))
    | RunnableLambda(lambda result, config: parse_baseline_values(result, config))
    | medical_prompt
    | llm_2
)

In [11]:
age = int(input("Enter the user's age :"))
sex = input("Enter the user's sex (male/female) :")
weight = int(input("Enter your weight in kg :"))
height = int(input("Enter your height in cm :"))
conditions = input("Enter your conditions (comma-separated):")
activity_level = input("Enter your activity level:")
medications = input("Enter your medications (comma-separated):")
input_data = {
    "query": f"What are the ideal nutritional requirements for a {age} year old {sex}, weighing {weight} kg, {height} cm tall?",
    "conditions": conditions,
    "activity_level": activity_level,
    "weight_kg": weight,
    "height_cm": height,
    "medications": medications
}

# Invoke the chain with the input data in the configurable context
result = chain.invoke(input_data, config={"configurable": {"input_data": input_data}})
print("Final Adjusted Nutritional Intake:")
print(result.content)

Enter the user's age :20
Enter the user's sex (male/female) :male
Enter your weight in kg :61
Enter your height in cm :175
Enter your conditions (comma-separated):iron deficiency, vitamin d deficiency
Enter your activity level:sedentary
Enter your medications (comma-separated):iron supplements


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Final Adjusted Nutritional Intake:
| Nutrient            | Baseline         | Adjusted         | Change         | Clinical Rationale                                                                                  | Risk Level                |
|---------------------|------------------|------------------|---------------|-----------------------------------------------------------------------------------------------------|---------------------------|
| Calcium             | 1000 mg/d        | 1000 mg/d        | None          | No direct adjustment for iron or vitamin D deficiency; maintain RDA[3][4].                          | Safe                      |
| Chromium            | 35 μg/d          | 35 μg/d          | None          | No evidence for adjustment in iron or vitamin D deficiency[3][4].                                   | Safe                      |
| Copper              | 900 μg/d         | 900 μg/d         | None          | No adjustment needed; monitor if high-dose iron long-t